In [2]:
from openai import OpenAI
import os
client = OpenAI(
    api_key=''
)

In [ ]:
# 生成回答
import json
import ast
# read training files
with open('training_data/train_1m.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

tot_prompt = "Imagine five different experts answering this question. All experts will write down 1 step of their thinking and then share it with the group. Then all experts will go on to the next step, etc. If any expert realizes they're wrong at any point, then they leave. And show me the result in the end. \nThe question is:\n\nYou are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

cot_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. Think step by step and make sure, then tell the reason. The movies should be before 2001.\n"

plain_prompt = "You are a recommender system. Given the user profile and historical data, analyze the user's interests and try to recommend movies the user might like. Your task is to add at least ten movies between the historical data and the target movie to connect them as an influence path. Then recommend them to the user one by one. Any adjacent movies should have a strong relation with each other, and make sure the movies are not included in the historical data. The movies should be before 2001.\n"

analysis_prompt = "You are a user analyzer. Given the user profile and historical liked movies, analyze the user's interests."

# suf_lst = ['','_CoT','_ToT']
suf_lst = ['_COT']
# prompt_lst = [plain_prompt, cot_prompt, tot_prompt]
prompt_lst = [cot_prompt]

count = 0
for _suf, _prompt in zip(suf_lst, prompt_lst):
    print(_suf)
    
    final_result = []
    for target, his_tra in content["llm"]:
        count+=1
        print(count)
        
        each_result = []
        for i in range(2):
            output = {}
            prompt = f"Historical data: {his_tra}\nTarget movie: {target}"
            
            message = [
                {"role": "system", "content": _prompt},
                {"role": "user", "content": prompt}
            ]
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message
            )
            
            first_reply = completion.choices[0].message.content
            message.append({"role": "assistant", "content": first_reply})
            message.append({"role": "user", "content": "Output the influence path in the format of python list object. Example output:\n[movie name1,\nmovie name2,\nmovie name3,\nmovie name4,\n...\ntarget movie name]\n"})
            completion = client.chat.completions.create(
                model="gpt-3.5-turbo",
                messages=message
            )
            
            influence_path = completion.choices[0].message.content
            
            try:
                influence_path_get = influence_path[influence_path.index("["):influence_path.index("]")+1]
                influence_path_lst = ast.literal_eval(influence_path_get)
            except:
                print(f"GPT输出不合规: First reply:\n {first_reply}, Second:\n {influence_path}")
                influence_path_lst = []
            
            each_result.append({'first': first_reply, 'second': influence_path, 'list': influence_path_lst})
            
        final_result.append(each_result)
        with open(f"user_study/examples.json", "w", encoding='utf-8') as fj:
            json.dump(final_result, fj)

In [3]:
# 生成解释
import json
import ast
from tqdm import tqdm
# read training files
with open('training_data/train_1m.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

analysis_prompt = "You are a user analyzer. Given the user profile and historical liked movies, analyze the user's interests."

final_result = []
for target, his_tra in tqdm(content["llm"]):
    
    each_result = []
    output = {}
    prompt = f"Historical liked movies: {his_tra}"
    
    message = [
        {"role": "system", "content": analysis_prompt},
        {"role": "user", "content": prompt}
    ]
    completion = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=message
    )
    
    first_reply = completion.choices[0].message.content
    final_result.append({"his_seq": his_tra, "target": target, "analysis": first_reply})
    
    with open(f"user_study/user_analysis.json", "w", encoding='utf-8') as f:
        json.dump(final_result, f)

100%|██████████| 123/123 [07:00<00:00,  3.42s/it]


In [2]:
import json
with open('user_study/movieLensTitles.json', 'r', encoding='utf-8') as f:
    movie_titles = json.load(f)
    
with open('user_study/outputIRS.json', 'r', encoding='utf-8') as f:
    content = json.load(f)

final_result = []
for path, _ in content:
    final_list = [movie_titles[x] for x in path]
    final_result.append({'list': path, 'final_list': final_list})

with open(f"user_study/outputIRSv2.json", "w", encoding='utf-8') as fj:
    json.dump(final_result, fj)

In [25]:
# 生成问卷
import json
import random

with open('user_study/output.json', 'r', encoding='utf-8') as f:
    gpt_content = json.load(f)
with open('user_study/outputIRSv2.json', 'r', encoding='utf-8') as f:
    irs_content = json.load(f)

def just_genre(seq):
    movie_title_seq = [[_[:_.index(' Genre:')], _[_.index(' Genre:'):]] if ' Genre:' in _ else [_, '']for _ in seq]
    offset = max([_[0] for _ in movie_title_seq], key=len, default="")
    return [_[0].ljust(len(offset)+5) + _[1] for _ in movie_title_seq]

answer = []
nn = 1
with open('user_study/user_study.json', 'w', encoding='utf-8') as fi:
    for [A, B], I in zip(gpt_content, irs_content):
        gpt_pathA = just_genre(A['final_list'])
        gpt_pathB = just_genre(B['final_list'])
        irs_path = just_genre(I['final_list'][:-1])
        
        if len(gpt_pathA) > 5:
            gpt_list = gpt_pathA
        elif len(gpt_pathB) > 5:
            gpt_list = gpt_pathB
        else:
            continue
        
        if len(irs_path) < 5:
            continue
        
        gpt_list += (25-len(gpt_list)) * [""]
        gpt_list = gpt_list[:12]
        
        irs_path += (25-len(irs_path)) * [""]
        irs_path = irs_path[:12]
        
        
        if len(I['final_list']) < 25:
            fi.write('User:'+str(nn)+'\n')
            prompt_content = A['prompt'].replace('|', ',').split('\n')
            prompt_content = just_genre(prompt_content)
            for line in prompt_content:
                fi.write(line+'\n')
            fi.write("\n\n")
            # print(A['prompt'])
            # print("\n\n")
    
            x = random.choice([True,False])  # True 左边是gpt
            answer.append(x)
            if x:
                offset = max(gpt_list, key=len, default="")
                for gpt_item, irs_item in zip(gpt_list, irs_path):
                    irs_item = irs_item.replace("|", ",")
                    fi.write(f'{gpt_item.ljust(len(offset)+5)} {irs_item}\n')
            else:
                offset = max(irs_path, key=len, default="")
                for gpt_item, irs_item in zip(gpt_list, irs_path):
                    irs_item = irs_item.replace("|", ",")
                    fi.write(f'{irs_item.ljust(len(offset)+5)} {gpt_item}\n')
                    
            fi.write("\n\n\n\n")
            # break
        
        # print(nn)
        nn += 1

with open('user_study/user_study_answer.json', 'w', encoding='utf-8') as fa:
    json.dump(answer, fa)

In [39]:
# 检查AB test user study 结果
import json
test_result_1 = 'abbaabbaab' + 'baaaabaacc' + 'abbcbabaaa'
test_result_3 = 'bacbcbbcab' + 'cbabccccbb' + 'ababcabccb' + 'cbccbbccaa'
test_result_2 = 'cabbccaaab' + 'bcababacab' + 'bacbcacaaa' + 'bbbabbbacc'

# test_result = test_result_1

with open('user_study/user_study_answer.json', 'r', encoding='utf-8') as fa:
    answer = json.load(fa)
    answer = ['a' if _ else 'b' for _ in answer]

for test_result in [test_result_1, test_result_2, test_result_3]:
    acc = []
    unpredict = 0
    for _ in range(30):
        if test_result[_] == 'c':
            unpredict += 1
        elif test_result[_] == answer[_]:
            acc.append(1)
        else:
            acc.append(0)
    
    print(sum(acc) / len(acc))
    print(unpredict/(len(acc)+unpredict))

0.6666666666666666
0.1
0.6363636363636364
0.26666666666666666
0.8421052631578947
0.36666666666666664


In [23]:
with open('user_study/user_study_answer.json', 'r', encoding='utf-8') as fa:
    answer = json.load(fa)
    answer = ['a' if _ else 'b' for _ in answer]
''.join(answer)
answer = 'baabbbbaab' + 'baaabbaaab' + 'ababbabbab' + 'bbaabbbaba'

'baabbbbaabbaaabbaaabababbabbabbbaabbbababbbbababaaaaaaabaaabbabaabbbaababababaabbaa'

我做了一个用户研究，给出30个样例以及baseline的结果，和我们提出的方法给出的结果，让志愿者进行判断。其中‘a’代表我们的方法好，‘b’代表baseline的方法好，‘c’则代表无法判断。
一共三个用户，给出的判断如下：
用户一：'abbaabbaab' + 'baaaabaacc' + 'abbcbabaaa'
用户二：'bacbcbbcab' + 'cbabccccbb' + 'ababcabccb'
用户三：'cabbccaaab' + 'bcababacab' + 'bacbcacaaa'

正确答案为：'baabbbbaab' + 'baaabbaaab' + 'ababbabbab'
请帮我分析每个用户的选择，以及利用卡方对三个结果进行一致性和置信度检验

In [45]:
from sklearn.metrics import cohen_kappa_score

def wash(lista, listb):
    output1 = []
    output2 = []
    
    for x, y in zip(lista, listb):
        if x == 'c' or y == 'c':
            output1.append(x)
            output2.append(x)
        else:
            output1.append(x)
            output2.append(y)
    
    return output1, output2

volunteer1 = 'bbbbbaaaaaaaaabaaaccaabcaaabab'
volunteer2 = 'aacacaacaacbabccccbaaaaacaacca'
volunteer3 = 'cabaccbaaaacabbaacaabbcacacbab'

a, b = wash(list(volunteer1), list(volunteer2))
kappa_12 = cohen_kappa_score(a, b)

a, b = wash(list(volunteer1), list(volunteer3))
kappa_13 = cohen_kappa_score(a, b)

a, b = wash(list(volunteer2), list(volunteer3))
kappa_23 = cohen_kappa_score(a, b)

print("Kappa between Volunteer 1 and 2:", kappa_12)
print("Kappa between Volunteer 1 and 3:", kappa_13)
print("Kappa between Volunteer 2 and 3:", kappa_23)

Kappa between Volunteer 1 and 2: 0.542483660130719
Kappa between Volunteer 1 and 3: 0.6428571428571428
Kappa between Volunteer 2 and 3: 0.72875226039783


In [ ]:
|